In [6]:
import os
import sys
import csv
import ccxt  # noqa: E402

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, data):
    with open(filename, mode='w') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows([['date', 'open', 'high', 'low', 'close', 'volume']])
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)

scrape_candles_to_csv('binance.csv', 'binance', 3, 'BTC/USDT', '1h', '2000-01-01T00:00:00Z', 100)

100 candles in total from 2020-01-17T14:00:00.000Z to 2020-01-21T17:00:00.000Z
200 candles in total from 2020-01-13T10:00:00.000Z to 2020-01-21T17:00:00.000Z
300 candles in total from 2020-01-09T06:00:00.000Z to 2020-01-21T17:00:00.000Z
400 candles in total from 2020-01-05T02:00:00.000Z to 2020-01-21T17:00:00.000Z
500 candles in total from 2019-12-31T22:00:00.000Z to 2020-01-21T17:00:00.000Z
600 candles in total from 2019-12-27T18:00:00.000Z to 2020-01-21T17:00:00.000Z
700 candles in total from 2019-12-23T14:00:00.000Z to 2020-01-21T17:00:00.000Z
800 candles in total from 2019-12-19T10:00:00.000Z to 2020-01-21T17:00:00.000Z
900 candles in total from 2019-12-15T06:00:00.000Z to 2020-01-21T17:00:00.000Z
1000 candles in total from 2019-12-11T02:00:00.000Z to 2020-01-21T17:00:00.000Z
1100 candles in total from 2019-12-06T22:00:00.000Z to 2020-01-21T17:00:00.000Z
1200 candles in total from 2019-12-02T18:00:00.000Z to 2020-01-21T17:00:00.000Z
1300 candles in total from 2019-11-28T14:00:00.00

10500 candles in total from 2018-11-10T06:00:00.000Z to 2020-01-21T17:00:00.000Z
10600 candles in total from 2018-11-06T02:00:00.000Z to 2020-01-21T17:00:00.000Z
10700 candles in total from 2018-11-01T22:00:00.000Z to 2020-01-21T17:00:00.000Z
10800 candles in total from 2018-10-28T18:00:00.000Z to 2020-01-21T17:00:00.000Z
10900 candles in total from 2018-10-24T14:00:00.000Z to 2020-01-21T17:00:00.000Z
11000 candles in total from 2018-10-20T10:00:00.000Z to 2020-01-21T17:00:00.000Z
11100 candles in total from 2018-10-16T06:00:00.000Z to 2020-01-21T17:00:00.000Z
11200 candles in total from 2018-10-12T02:00:00.000Z to 2020-01-21T17:00:00.000Z
11300 candles in total from 2018-10-07T22:00:00.000Z to 2020-01-21T17:00:00.000Z
11400 candles in total from 2018-10-03T18:00:00.000Z to 2020-01-21T17:00:00.000Z
11500 candles in total from 2018-09-29T14:00:00.000Z to 2020-01-21T17:00:00.000Z
11600 candles in total from 2018-09-25T10:00:00.000Z to 2020-01-21T17:00:00.000Z
11700 candles in total from 

20700 candles in total from 2017-09-12T10:00:00.000Z to 2020-01-21T17:00:00.000Z
20800 candles in total from 2017-09-08T06:00:00.000Z to 2020-01-21T17:00:00.000Z
20900 candles in total from 2017-09-04T02:00:00.000Z to 2020-01-21T17:00:00.000Z
21000 candles in total from 2017-08-30T22:00:00.000Z to 2020-01-21T17:00:00.000Z
21100 candles in total from 2017-08-26T18:00:00.000Z to 2020-01-21T17:00:00.000Z
21200 candles in total from 2017-08-22T14:00:00.000Z to 2020-01-21T17:00:00.000Z
21300 candles in total from 2017-08-18T10:00:00.000Z to 2020-01-21T17:00:00.000Z
21400 candles in total from 2017-08-17T04:00:00.000Z to 2020-01-21T17:00:00.000Z
Saved 21400 candles from 2017-08-17T04:00:00.000Z to 2020-01-21T17:00:00.000Z to binance.csv
